Framework: TensorFlow2 + Keras (можно використовувати Pytorch)

1. Повнозв'язані нейронні мережі
Вирішіть завдання класифікації даних, з якими ви працювали в лабораторній № 1 за допомогою повнозв’язаної нейромережі прямого поширення (fully connected feed-forward network). Результати порівняйте з одержаними раніше. 

In [10]:
import pandas as pd

# Завантаження датасету
file_path = 'USA_cars_datasets.csv'
data = pd.read_csv(file_path)

# Відображення перших кількох рядків для аналізу структури даних
data.head(), data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2499 entries, 0 to 2498
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    2499 non-null   int64  
 1   price         2499 non-null   int64  
 2   brand         2499 non-null   object 
 3   model         2499 non-null   object 
 4   year          2499 non-null   int64  
 5   title_status  2499 non-null   object 
 6   mileage       2499 non-null   float64
 7   color         2499 non-null   object 
 8   vin           2499 non-null   object 
 9   lot           2499 non-null   int64  
 10  state         2499 non-null   object 
 11  country       2499 non-null   object 
 12  condition     2499 non-null   object 
dtypes: float64(1), int64(4), object(8)
memory usage: 253.9+ KB


(   Unnamed: 0  price      brand    model  year   title_status   mileage  \
 0           0   6300     toyota  cruiser  2008  clean vehicle  274117.0   
 1           1   2899       ford       se  2011  clean vehicle  190552.0   
 2           2   5350      dodge      mpv  2018  clean vehicle   39590.0   
 3           3  25000       ford     door  2014  clean vehicle   64146.0   
 4           4  27700  chevrolet     1500  2018  clean vehicle    6654.0   
 
     color                  vin        lot       state country      condition  
 0   black    jtezu11f88k007763  159348797  new jersey     usa   10 days left  
 1  silver    2fmdk3gc4bbb02217  166951262   tennessee     usa    6 days left  
 2  silver    3c4pdcgg5jt346413  167655728     georgia     usa    2 days left  
 3    blue    1ftfw1et4efc23745  167753855    virginia     usa  22 hours left  
 4     red    3gcpcrec2jg473991  167763266     florida     usa  22 hours left  ,
 None)

In [11]:
import numpy as np

# Визначимо категорії для price: низький, середній, високий
# Використовуємо квантильний розподіл
data['price_category'] = pd.qcut(data['price'], q=3, labels=['low', 'medium', 'high'])

# Видалимо неінформативні колонки
data_cleaned = data.drop(columns=['Unnamed: 0', 'vin', 'lot', 'country'])

# Перевіримо розподіл цільової змінної та новий датасет
price_distribution = data['price_category'].value_counts()
data_cleaned.head(), price_distribution


(   price      brand    model  year   title_status   mileage   color  \
 0   6300     toyota  cruiser  2008  clean vehicle  274117.0   black   
 1   2899       ford       se  2011  clean vehicle  190552.0  silver   
 2   5350      dodge      mpv  2018  clean vehicle   39590.0  silver   
 3  25000       ford     door  2014  clean vehicle   64146.0    blue   
 4  27700  chevrolet     1500  2018  clean vehicle    6654.0     red   
 
         state      condition price_category  
 0  new jersey   10 days left            low  
 1   tennessee    6 days left            low  
 2     georgia    2 days left            low  
 3    virginia  22 hours left           high  
 4     florida  22 hours left           high  ,
 low       834
 medium    833
 high      832
 Name: price_category, dtype: int64)

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Вибір числових і категоріальних колонок
numeric_columns = ['year', 'mileage']
categorical_columns = ['brand', 'model', 'title_status', 'color', 'state', 'condition']

# Нормалізація числових колонок
scaler = StandardScaler()
data_cleaned[numeric_columns] = scaler.fit_transform(data_cleaned[numeric_columns])

# One-hot encoding для категоріальних змінних
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
categorical_encoded = encoder.fit_transform(data_cleaned[categorical_columns])
categorical_encoded_df = pd.DataFrame(categorical_encoded, columns=encoder.get_feature_names_out(categorical_columns))

# Об'єднання всіх ознак
features = pd.concat([data_cleaned[numeric_columns], categorical_encoded_df], axis=1)
target = data_cleaned['price_category']

# Розділення на тренувальний і тестовий набори
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42, stratify=target)

# Перевіримо розміри отриманих наборів
X_train.shape, X_test.shape, y_train.shape, y_test.shape


c:\Users\denis\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


((1999, 299), (500, 299), (1999,), (500,))

Тренувальний набір: 1999 прикладів із 299 ознаками.
Тестовий набір: 500 прикладів із 299 ознаками.

In [13]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Закодуємо цільову змінну для нейронної мережі
y_train_encoded = pd.get_dummies(y_train).values
y_test_encoded = pd.get_dummies(y_test).values

# Побудова моделі
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(3, activation='softmax')  # 3 класи (low, medium, high)
])

# Компіляція моделі
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Навчання моделі
history = model.fit(X_train, y_train_encoded, 
                    validation_data=(X_test, y_test_encoded), 
                    epochs=50, 
                    batch_size=32, 
                    verbose=1)


Epoch 1/50
63/63 [==============================] - 2s 8ms/step - loss: 0.9631 - accuracy: 0.5503 - val_loss: 0.8076 - val_accuracy: 0.6900
Epoch 2/50
63/63 [==============================] - 0s 4ms/step - loss: 0.7231 - accuracy: 0.7044 - val_loss: 0.6359 - val_accuracy: 0.7860
Epoch 3/50
63/63 [==============================] - 0s 6ms/step - loss: 0.5873 - accuracy: 0.7699 - val_loss: 0.5824 - val_accuracy: 0.7820
Epoch 4/50
63/63 [==============================] - 0s 5ms/step - loss: 0.5074 - accuracy: 0.8049 - val_loss: 0.5460 - val_accuracy: 0.7940
Epoch 5/50
63/63 [==============================] - 0s 5ms/step - loss: 0.4545 - accuracy: 0.8199 - val_loss: 0.5356 - val_accuracy: 0.7880
Epoch 6/50
63/63 [==============================] - 0s 6ms/step - loss: 0.4033 - accuracy: 0.8429 - val_loss: 0.5371 - val_accuracy: 0.7940
Epoch 7/50
63/63 [==============================] - 0s 5ms/step - loss: 0.3844 - accuracy: 0.8479 - val_loss: 0.5443 - val_accuracy: 0.7840
Epoch 8/50
63/63 [==

2. Згорткові нейронні мережі
Вирішіть завдання класифікації зображень за допомогою згорткової (convolutional) нейромережі двома способами
а) навчить мережу з нуля (from scratch)
б) застосуйте перенесення навчання (transfer learning from pre-trained weights)
Порівняйте результати (якщо в обраному датасеті класів забагато, достатньо залишити 3-5).

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Завантаження даних
train_data = pd.read_csv('fashion-mnist_train.csv')  
test_data = pd.read_csv('fashion-mnist_test.csv')   

# Відокремлення ознак і міток
X_train = train_data.drop('label', axis=1).values
y_train = train_data['label'].values

X_test = test_data.drop('label', axis=1).values
y_test = test_data['label'].values

# Перетворення табличних даних у формат зображень
X_train = X_train.reshape(-1, 28, 28, 1) / 255.0  # Масштабування значень у діапазон [0, 1]
X_test = X_test.reshape(-1, 28, 28, 1) / 255.0

# One-hot кодування 
encoder = OneHotEncoder(sparse=False)
y_train = encoder.fit_transform(y_train.reshape(-1, 1))
y_test = encoder.transform(y_test.reshape(-1, 1))

# Побудова моделі
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')  # 10 класів
])

# Компіляція моделі
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Навчання моделі
history = model.fit(X_train, y_train, 
                    validation_data=(X_test, y_test), 
                    epochs=10, 
                    batch_size=32)

# Оцінка моделі
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.2f}")

c:\Users\denis\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch 1/10
1875/1875 [==============================] - 55s 27ms/step - loss: 0.5978 - accuracy: 0.7806 - val_loss: 0.3770 - val_accuracy: 0.8488
Epoch 2/10
1875/1875 [==============================] - 63s 34ms/step - loss: 0.4113 - accuracy: 0.8499 - val_loss: 0.3012 - val_accuracy: 0.8877
Epoch 3/10
1875/1875 [==============================] - 67s 36ms/step - loss: 0.3666 - accuracy: 0.8671 - val_loss: 0.2760 - val_accuracy: 0.8966
Epoch 4/10
1875/1875 [==============================] - 102s 54ms/step - loss: 0.3409 - accuracy: 0.8755 - val_loss: 0.2641 - val_accuracy: 0.9021
Epoch 5/10
1875/1875 [==============================] - 68s 36ms/step - loss: 0.3273 - accuracy: 0.8802 - val_loss: 0.2523 - val_accuracy: 0.9044
Epoch 6/10
1875/1875 [==============================] - 59s 32ms/step - loss: 0.3110 - accuracy: 0.8876 - val_loss: 0.2392 - val_accuracy: 0.9113
Epoch 7/10
1875/1875 [==============================] - 60s 32ms/step - loss: 0.3009 - accuracy: 0.8884 - val_loss: 0.2546 

In [17]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input, Conv2D, Resizing

# Відокремлення ознак і міток
X_train = train_data.drop('label', axis=1).values
y_train = train_data['label'].values

X_test = test_data.drop('label', axis=1).values
y_test = test_data['label'].values

# Перетворення табличних даних у формат зображень
X_train = X_train.reshape(-1, 28, 28, 1) / 255.0  # Масштабування значень у діапазон [0, 1]
X_test = X_test.reshape(-1, 28, 28, 1) / 255.0

# One-hot кодування міток
encoder = OneHotEncoder(sparse=False)
y_train = encoder.fit_transform(y_train.reshape(-1, 1))
y_test = encoder.transform(y_test.reshape(-1, 1))

# Попередньо натренована модель
base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=Input(shape=(32, 32, 3)))

# Перетворення зображень 28x28x1 у формат 32x32x3 для сумісності
resize_layer = Sequential([
    Resizing(32, 32),  # Зміна розміру
    Conv2D(3, (1, 1))  # Конвертація в 3 канали
])

X_train_resized = resize_layer(X_train)
X_test_resized = resize_layer(X_test)

# Замороження ваг базової моделі
base_model.trainable = False

# Додавання власних шарів
model = Sequential([
    base_model,
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')  # 10 класів
])

# Компіляція моделі
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Навчання моделі
history = model.fit(X_train_resized, y_train, 
                    validation_data=(X_test_resized, y_test), 
                    epochs=10, 
                    batch_size=32)

# Оцінка моделі
test_loss, test_accuracy = model.evaluate(X_test_resized, y_test)
print(f"Test Accuracy (Transfer Learning): {test_accuracy:.2f}")

c:\Users\denis\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


9406464/9406464 [==============================] - 5s 1us/step
Epoch 1/10
1875/1875 [==============================] - 87s 44ms/step - loss: 1.1113 - accuracy: 0.6187 - val_loss: 0.8889 - val_accuracy: 0.6813
Epoch 2/10
1875/1875 [==============================] - 93s 49ms/step - loss: 0.9217 - accuracy: 0.6745 - val_loss: 0.8405 - val_accuracy: 0.6993
Epoch 3/10
1875/1875 [==============================] - 86s 46ms/step - loss: 0.8847 - accuracy: 0.6862 - val_loss: 0.8143 - val_accuracy: 0.7045
Epoch 4/10
1875/1875 [==============================] - 89s 47ms/step - loss: 0.8644 - accuracy: 0.6942 - val_loss: 0.8037 - val_accuracy: 0.7111
Epoch 5/10
1875/1875 [==============================] - 90s 48ms/step - loss: 0.8473 - accuracy: 0.6985 - val_loss: 0.7857 - val_accuracy: 0.7186
Epoch 6/10
1875/1875 [==============================] - 96s 51ms/step - loss: 0.8386 - accuracy: 0.7012 - val_loss: 0.7806 - val_accuracy: 0.7171
Epoch 7/10
1875/1875 [==============================] - 93s 4

3. Рекурентні нейронні мережі
Вирішіть задачу класифікації текстів (з якими ви працювали в лабораторній № 2) за допомогою рекурентної нейромережі двома способами:
а) навчить мережу і embedding шар з нуля (from scratch)
б) використовуючи pretrained word embeddings
 Результати порівняйте між собою і з одержаними раніш. 

In [19]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Завантаження даних
data = pd.read_csv('IMDB Dataset.csv')

# Розділення на тренувальний і тестовий набори
X_train, X_test, y_train, y_test = train_test_split(data['review'], data['sentiment'], test_size=0.2, random_state=42)

# Параметри для обробки тексту
max_words = 10000  # Максимальна кількість слів у словнику
max_len = 200      # Максимальна довжина тексту

# Токенізація тексту
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

# Перетворення тексту в послідовності
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Заповнення послідовностей до однакової довжини
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post', truncating='post')

# Перетворення міток на числовий формат
y_train = (y_train == "positive").astype(int)
y_test = (y_test == "positive").astype(int)

# Побудова RNN моделі
model = Sequential([
    Embedding(input_dim=max_words, output_dim=128, input_length=max_len),
    LSTM(64, return_sequences=False),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Компіляція моделі
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Навчання моделі
history = model.fit(X_train_pad, y_train, validation_data=(X_test_pad, y_test), epochs=5, batch_size=32)

# Оцінка моделі
test_loss, test_accuracy = model.evaluate(X_test_pad, y_test)
print(f"Test Accuracy: {test_accuracy:.2f}")


Epoch 1/5
1250/1250 [==============================] - 190s 149ms/step - loss: 0.6705 - accuracy: 0.5768 - val_loss: 0.5797 - val_accuracy: 0.7165
Epoch 2/5
1250/1250 [==============================] - 223s 179ms/step - loss: 0.6269 - accuracy: 0.6413 - val_loss: 0.5623 - val_accuracy: 0.7145
Epoch 3/5
1250/1250 [==============================] - 217s 174ms/step - loss: 0.5648 - accuracy: 0.7171 - val_loss: 0.6745 - val_accuracy: 0.5454
Epoch 4/5
1250/1250 [==============================] - 213s 171ms/step - loss: 0.4381 - accuracy: 0.7994 - val_loss: 0.3348 - val_accuracy: 0.8614
Epoch 5/5
313/313 [==============================] - 18s 56ms/step - loss: 0.3158 - accuracy: 0.8717
Test Accuracy: 0.87


In [22]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from gensim.models import KeyedVectors

# Завантаження даних
data = pd.read_csv('IMDB Dataset.csv')

# Розділення на тренувальний і тестовий набори
X_train, X_test, y_train, y_test = train_test_split(data['review'], data['sentiment'], test_size=0.2, random_state=42)

# Параметри для обробки тексту
max_words = 10000  # Максимальна кількість слів у словнику
max_len = 200      # Максимальна довжина тексту

# Токенізація тексту
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

# Перетворення тексту в послідовності
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Заповнення послідовностей до однакової довжини
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post', truncating='post')

# Перетворення міток на числовий формат
y_train = (y_train == "positive").astype(int)
y_test = (y_test == "positive").astype(int)

# Завантаження FastText
print("Завантаження попередньо натренованих ембедингів...")
fasttext_model = KeyedVectors.load_word2vec_format('https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz', binary=False)

# Побудова матриці ембедингів
embedding_dim = 300  # Розмірність векторів FastText
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i < max_words:
        if word in fasttext_model:
            embedding_matrix[i] = fasttext_model[word]

# Побудова RNN моделі з попередньо натренованими ембедингами
model = Sequential([
    Embedding(input_dim=max_words, output_dim=embedding_dim,
              weights=[embedding_matrix],
              input_length=max_len,
              trainable=False),  # Попередньо натреновані ваги не оновлюються
    LSTM(64, return_sequences=False),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Компіляція моделі
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Навчання моделі
history = model.fit(X_train_pad, y_train, validation_data=(X_test_pad, y_test), epochs=5, batch_size=32)

# Оцінка моделі
test_loss, test_accuracy = model.evaluate(X_test_pad, y_test)
print(f"Test Accuracy with FastText: {test_accuracy:.2f}")


Завантаження попередньо натренованих ембедингів...
Epoch 1/5
1250/1250 [==============================] - 307s 242ms/step - loss: 0.6857 - accuracy: 0.5466 - val_loss: 0.6464 - val_accuracy: 0.6610
Epoch 2/5
1250/1250 [==============================] - 269s 215ms/step - loss: 0.6809 - accuracy: 0.5571 - val_loss: 0.6801 - val_accuracy: 0.5625
Epoch 3/5
1250/1250 [==============================] - 261s 209ms/step - loss: 0.6853 - accuracy: 0.5427 - val_loss: 0.6884 - val_accuracy: 0.5519
Epoch 4/5
1250/1250 [==============================] - 259s 207ms/step - loss: 0.6830 - accuracy: 0.5570 - val_loss: 0.6672 - val_accuracy: 0.6181
Epoch 5/5
313/313 [==============================] - 29s 93ms/step - loss: 0.6914 - accuracy: 0.5198
Test Accuracy with FastText: 0.52
